# Proyecto Final ML23

In [1]:
import numpy as np
import random
import gymnasium as gym
import time

# **Parametros**

* `interacion:`       **int**   #Episodios
* `learning_rate:`    **float** #Learning rate
* `gamma:`            **float** #Discount rate
* `epsilon:`          **float** # Exploration probability
* `seed:`             **int**   # Define a seed so that we get reproducible results
* `n_runs:`           **int**   # Number of runs
* `action_size:`      **int**   # Number of possible actions
* `state_size:`       **int**   # Number of possible states
* `proba_frozen:`     **float** # Probability that a tile is frozen
 


In [ ]:
seed = "waf"
rng = np.random.default(seed)

In [ ]:
class Qlearning:
    def _init_(self, learning_rate, gamma, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.reset_qtable()
    
    def update(self, state, action, reward, new_state):
        #Update Q(s,a) = Q(s,a) + Lr[ R(s,a) + gamma * maxQ(s',a') - Q(s,a)]
        delta = (
            reward
            + self.gamma *np.max(self.qtable[new_state, :])
            - self.qtable[state, action]
        )
        q_update = self.qtable[state, action] + self.learning_rate * delta
        return q_update
    
    def reset_qtable(self):
        #Resets Q-Table
        self.qtable=np.zeros((self.state_size,self.action_size))

class EpsilonGreedy:
    def _init_(self, epsilon):
        self.epsilon = epsilon

    def choose_action(self, action_space, state, qtable):
        "Escoger una accion 'a' en el contexto actual (s)"
        #Randomiza un numero
        explore_exploit_tradeoff = rng.uniform(0,1)

        #Exploracion
        if(explore_exploit_tradeoff<self.epsilon):
            action = action_space.sample()
        
        #Explotacion (tomar el Q-valor mas grande para este estado)
        else:
            if np.all(qtable[state, :]) == qtable [state, 0]:
                action = action_space.sample()
            else:
                action = np.argmax(qtable[state,:])
        return action



In [2]:
from gym.envs.toy_text.frozen_lake import generate_random_map #Utilizar esto para generar mapas y evaluar al agente entrenado


def run():
    env = gym.make('FrozenLake-v1', desc=None, render_mode="human" ,map_name="4x4", is_slippery=False)
    
    lr = 0.6 # alpha / learning rate
    discount_factor = 0.7 # gamma / discount factor
    
    state = env.reset()[0] #Estados 0 al 15, 0=top left, 15=bottom left 
    hoyo = False
    perdido = 200
    analisis = env.observation_space
    interacion = 0

    while(not hoyo):
        action = env.action_space.sample() #acciones 0=izq,1=abajo,2=der,3=arriba
        info = env.step(action)
        new_state,reward,hoyo,_,_= info
    
        state = new_state
        interacion = interacion + 1
        
        print("Interaccion numero: "+ str(interacion)+" hizo la accion numero: "+str(action))
        print(info)
        env.render()
        time.sleep(7)
        
    env.close()
    
    print(analisis)
run() #corre el Frozen Lake

Interacion numero: 1 hizo la accion numero: 1
(4, 0.0, False, False, {'prob': 1.0})
Interacion numero: 2 hizo la accion numero: 0
(4, 0.0, False, False, {'prob': 1.0})
Interacion numero: 3 hizo la accion numero: 1
(8, 0.0, False, False, {'prob': 1.0})
Interacion numero: 4 hizo la accion numero: 3
(4, 0.0, False, False, {'prob': 1.0})
Interacion numero: 5 hizo la accion numero: 2
(5, 0.0, True, False, {'prob': 1.0})
Discrete(16)


In [8]:
#Qtable Printout
qtable = np.zeros((16,4))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
